# SimEnv Demo

This notebook is intended to show the functionality of the SimEnv. 

In [ ]:
using Dare


## Simplest grid

In [ ]:
CM = [ 0  0  1
       0  0  2
      -1 -2  0];

An entry in `CM[2,3]`.